In [1]:
import os
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data
from PIL import Image
import pickle
from tqdm import tqdm

# Load train data

In [2]:
train_dir = '/home/featurize/data/tiny-imagenet-200/train/'
train_dataset = datasets.ImageFolder(
    train_dir) # 
train_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8)
toTensor = transforms.ToTensor()
toPIL = transforms.ToPILImage()
# print("加载训练目录: %s" % (train_dir))
# print("加载类别数量 %d"%len(train_dataset.classes))
# print("类别 n01443537 映射 index %d" % train_dataset.class_to_idx['n01443537'])
# print("图片 %s 映射到 idx %d" % train_dataset.imgs[0])

## Process test data

In [3]:
val_imgs = []
val_labels = []

val_dir = '/home/featurize/data/tiny-imagenet-200/val/'
with open(os.path.join(val_dir, 'val_annotations.txt'), "r") as f:
    val_f_annot = [(s.split('\t')[0], s.split('\t')[1]) for s in f.readlines()]
    
for img_file, class_name in val_f_annot:
    img = np.array(Image.open(os.path.join(val_dir, 'images', img_file)))
    if len(img.shape) != 3:
        img = img.reshape(64, 64, 1).repeat(3, axis=2)
    val_imgs.append(img)
    val_labels.append(train_dataset.class_to_idx[class_name])
# from utils import SubTrainDataset
# test_dataset = SubTrainDataset(val_imgs, val_labels, transform=transforms.ToTensor())
pickle.dump((val_imgs, val_labels), open("/home/featurize/work/RAI2project/RAI2data/similarity/TinyImagenet_test.pkl", "wb"))

## Compute mean and std

In [4]:
X_tensor = []
for x, y in tqdm(train_dataset):
    X_tensor.append(toTensor(x))
X_tensor = torch.stack(X_tensor)

100%|██████████| 100000/100000 [00:20<00:00, 4849.20it/s]


In [5]:
# mean
X_tensor.mean(dim=[0, 2, 3]).numpy()

array([0.48023874, 0.4480645 , 0.39750546], dtype=float32)

In [6]:
# mean
X_tensor.std(dim=[0, 2, 3]).numpy()

array([0.27643642, 0.2688633 , 0.28158993], dtype=float32)

# Random split train dataset

In [7]:
np.random.seed(0)
vic_idx = np.random.choice(len(train_dataset), len(train_dataset) // 2, replace=False)

In [8]:
X_list = []
y_list = []
# recover from np.array
# from PIL import Image
# im = Image.fromarray(X_train_list[0])
for x, y in tqdm(train_dataset):
    X_list.append(np.array(x))
    y_list.append(y)

100%|██████████| 100000/100000 [00:11<00:00, 8446.32it/s]


In [9]:
X_np = np.array(X_list)
y_np = np.array(y_list)

In [10]:
X_set1 = X_np[vic_idx]
y_set1 = y_np[vic_idx]
X_set2 = X_np[~vic_idx]
y_set2 = y_np[~vic_idx]

In [11]:
pickle.dump(((X_set1, y_set1), (X_set2, y_set2)), open("/home/featurize/work/RAI2project/similarity/TinyImagenet_sub_train_split.pkl", "wb"))

## subset 1 mean and std

In [12]:
X_tensor[torch.tensor(vic_idx)].mean(dim=[0,2,3]).numpy()

array([0.480212  , 0.4481674 , 0.39781156], dtype=float32)

In [13]:
X_tensor[torch.tensor(vic_idx)].std(dim=[0,2,3]).numpy()

array([0.2763922 , 0.26874343, 0.28151527], dtype=float32)

## subset 2 mean and std

In [14]:
X_tensor[~torch.tensor(vic_idx)].mean(dim=[0,2,3]).numpy()

array([0.48034695, 0.44770736, 0.39680204], dtype=float32)

In [15]:
X_tensor[~torch.tensor(vic_idx)].std(dim=[0,2,3]).numpy()

array([0.2763423 , 0.26875827, 0.28141335], dtype=float32)